In [11]:
#Import Required Libraries:
import pandas as pd
import networkx as nx
import json

#Load Author Network Data
data = pd.read_csv(r"C:\Users\Rathn\Documents\DV Project\MajorAssignment3\data_scopus.csv")

nodes = data['EID'].values
data=data.fillna(0)
data.head(2)

,Title,Year,EID,Abstract,Publisher,Conference name,Conference date,Authors,Author(s) ID,Authors with affiliations,Source title,Abbreviated Source Title,Cited by
0,Virtual reality applications for the built env...,2020,2-s2.0-85086464158,With its advanced capabilities of immersive an...,Elsevier B.V.,0,0,"Zhang Y., Liu H., Kang S.-C., Al-Hussein M.",57202536919;53984550800;57215426514;6603541102;,"Zhang, Y., Department of Civil and Environment...",Automation in Construction,Autom Constr,0.0
1,"Self-tracking while doing sport: Comfort, moti...",2020,2-s2.0-85082875828,The spread of wearable technologies is paving ...,Academic Press,0,0,"Rapp A., Tirabeni L.",23398572100;57191836654;,"Rapp, A., Computer Science Department, Univers...",International Journal of Human Computer Studies,Int J Hum Comput Stud,0.0


In [12]:

data = data.dropna(subset=['Author(s) ID','Year','Authors','Authors with affiliations'])

# nodes = df['Author(s) ID'].values
data = data.fillna(0)
G = nx.Graph()
nodes = {}


for _, row in data.iterrows():
    # Extract authors from the "Authors" column
    authors = row['Authors'].split(',')
    author_Id = row['Author(s) ID'].split(';')
    Title=row['Title']
    Year=row['Year']
    Citations=row['Cited by']
    Publisher=row['Publisher']
    Authorwa=row['Authors with affiliations']

    # Add authors as nodes

    for author in range(len(authors)):

        aid= author_Id[author]
        aname= ';'.join(authors)
        title=Title

        if(aid!=""):
          #print(aid)
         nodes={'id':aid,
          "Authors": aname,
          "Title": title,
          "Year": Year,
          "Citations": Citations,
          "Publisher": Publisher,
          "Author with affiliations":Authorwa
           }
         G.add_node(aid,**nodes)

print(G)
from networkx.readwrite import json_graph
with open("publications_NetworkGraph.json", "w") as f:
    json.dump(json_graph.node_link_data(G), f)


Graph with 1553 nodes and 0 edges


In [13]:
# Load the data from the JSON file
with open('publications_NetworkGraph.json', 'r') as infile:
    data1 = json.load(infile)
newnodes=[]
# Create a NetworkX graph
G=nx.Graph()

def get_author_country(authors_with_affiliations):
    first_affiliation = authors_with_affiliations.split(';')[0].strip()
    country = first_affiliation.split(',')[-1].strip()
    return country
for node_data in data1['nodes']:
    eid = node_data.get('id')
    authors=node_data.get('Authors')
    title = node_data.get('Title')
    year = node_data.get('Year')
    citations = node_data.get('Citations')
    publisher = node_data.get('Publisher')

    authors_with_affiliations = str(node_data.get('Author with affiliations'))

    country = get_author_country(authors_with_affiliations)

    G.add_node(eid,authors=authors, title=title, year=year, citations=citations, publisher=publisher,authors_with_affiliations=authors_with_affiliations, country=country)
# Generate clusters based on authors' countries
clusters = list(nx.connected_components(G))

# Assign classes to each cluster
class_mapping = {node: idx for idx, cluster in enumerate(clusters) for node in cluster}
nx.set_node_attributes(G, class_mapping, 'class')

# title = data["Title"]
# authors_info = data["Authors with affiliations"].split(';')

# Create a dictionary to store co-authorship relationships
coauthorship_dict = {}
for row in data.iterrows():
    authors = row[1]['Author(s) ID'].split(';')
    for i in range(len(authors)):
      for j in range(i+1, len(authors)):
        if(authors[i]!="" and authors[j]!=""):
          G.add_edge(authors[i], authors[j])


print(G)
# print(edges)


# Save the data as a JSON file with proper character encoding
coAuthors_data_file = {'nodes': [{'id': node, **G.nodes[node]} for node in G.nodes()],'links': [{'source': source, 'target': target} for source, target in G.edges()]}
#coauthorship_data = {'nodes': [{'id': node, **G.nodes[node]} for node in G.nodes()],'edges': edges}
with open('coAuthors_data_file.json', 'w', encoding='utf-8') as outfile:
    json.dump(coAuthors_data_file, outfile, ensure_ascii=False)

Graph with 1553 nodes and 3049 edges
